In [1]:
import pyphen

In [2]:
pyphen.language_fallback("hu_HU")

'hu_HU'

In [3]:
'hu_HU' in pyphen.LANGUAGES

True

In [4]:
hypher = pyphen.Pyphen(lang='hu_HU')

In [5]:
hypher.inserted('almáspite')

'al-más-pi-te'

In [6]:
hypher.inserted('asszonnyal')

'asz-szony-nyal'

In [7]:
hypher.wrap("megszentségteleníthetetlenségeskedéseitekért", 30)

('megszentségteleníthetetlensé-', 'geskedéseitekért')

In [8]:
for pair in hypher.iterate("megszentségteleníthetetlenségeskedéseitekért"):
    print(pair)

('megszentségteleníthetetlenségeskedéseite', 'kért')
('megszentségteleníthetetlenségeskedései', 'tekért')
('megszentségteleníthetetlenségeskedése', 'itekért')
('megszentségteleníthetetlenségeskedé', 'seitekért')
('megszentségteleníthetetlenségeske', 'déseitekért')
('megszentségteleníthetetlenséges', 'kedéseitekért')
('megszentségteleníthetetlensé', 'geskedéseitekért')
('megszentségteleníthetetlen', 'ségeskedéseitekért')
('megszentségteleníthetet', 'lenségeskedéseitekért')
('megszentségteleníthe', 'tetlenségeskedéseitekért')
('megszentségtelenít', 'hetetlenségeskedéseitekért')
('megszentségtele', 'níthetetlenségeskedéseitekért')
('megszentségte', 'leníthetetlenségeskedéseitekért')
('megszentség', 'teleníthetetlenségeskedéseitekért')
('megszent', 'ségteleníthetetlenségeskedéseitekért')
('meg', 'szentségteleníthetetlenségeskedéseitekért')


In [9]:
hypher.inserted("megszentségteleníthetetlenségeskedéseitekért")

'meg-szent-ség-te-le-nít-he-tet-len-sé-ges-ke-dé-se-i-te-kért'

In [11]:
hypher.positions("almáspite")

[2, 5, 7]

In [48]:
def hyph_class(word, hypher=pyphen.Pyphen(lang='hu_HU'), aslist=False):
    """Hyphenating classification of the characters in the word. {B(egin),M(iddle),E(nd),S(ingle)}"""
    ret = list('M' * len(word))
    ret[0]='B'
    ret[-1]='E'
    for i in hypher.positions(word):
        ret[i]='B'
        if(ret[i-1]=='B'):
            ret[i-1]='S'
        else:
            ret[i-1]='E'
    if(aslist):
        return ret
    return "".join(ret)

In [49]:
hyph_class('leopárd',hypher)

'BESBMME'

In [43]:
hypher.inserted("leopárd")

'le-o-párd'

In [50]:
hyph_class("megszentségteleníthetetlenségeskedéseitekért")

'BMEBMMMEBMEBEBEBMEBEBMEBMEBEBMEBEBEBESBEBMME'